In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # Warnin 방지용
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl


from matplotlib import font_manager, rc

%matplotlib inline
sns.set(style='whitegrid')
plt.rcParams['figure.figsize']=(12,9)

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

In [3]:
import matplotlib.font_manager

matplotlib.font_manager._rebuild()
sorted([f.name for f in matplotlib.font_manager.fontManager.ttflist if f.name.startswith("Nanum")])

['NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumGothic',
 'NanumMyeongjo',
 'NanumMyeongjo',
 'NanumMyeongjo',
 'NanumMyeongjo',
 'NanumMyeongjo',
 'NanumMyeongjo']

In [4]:
# 폰트 설정
mpl.rc('font', family='NanumGothic')
# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

In [5]:
factor = pd.read_csv('data2/dw_fire_fighting.csv', encoding='ansi')
factor.head()

,fire_date,city,district,district_code,fire_type,type_code,fire_source,fire_source_code,fire_factor,fire_factor_code,casualties,dead,injury,property_damage,place,place_code
0,2019-01,부산광역시,부산진구,D07,"건축,구조물",T05,작동기기,S02,전기적 요인,F05,0,0,0,45,기타,P05
1,2019-01,부산광역시,남구,D04,"건축,구조물",T05,미상,S04,미상,F04,0,0,0,2997,공동주택,P02
2,2019-01,부산광역시,사상구,D09,"건축,구조물",T05,미상,S04,미상,F04,0,0,0,0,단독주택,P07
3,2019-01,부산광역시,강서구,D01,"건축,구조물",T05,"담뱃불, 라이터불",S07,부주의,F01,0,0,0,113,기타,P05
4,2019-01,부산광역시,강서구,D01,"건축,구조물",T05,작동기기,S02,전기적 요인,F05,0,0,0,65,단독주택,P07


In [6]:
# 월 단위
month = []
for m in factor['fire_date']:
    if m not in month:
        month.append(m)
        
# 월 화재건수
monthsum = []
for i in range(len(month)):
    m = factor[factor.fire_date == month[i]]
    ms = m['fire_date'].count()
    monthsum.append(ms)
    
# 월 피해규모
m_damagesum = []
for i in range(len(month)):
    m = factor[factor.fire_date == month[i]]
    ds = m['property_damage'].sum()
    m_damagesum.append(ds)



In [7]:
# 월별 화재건수
month_df = pd.DataFrame({
    'fire_date': month,
    'occur_cnt': monthsum
})

# 월별 피해규모
month_damage_df = pd.DataFrame({
    'fire_date' : month,
    'property_damage_sum_k' : m_damagesum
})

month_df

,fire_date,occur_cnt
0,2019-01,260
1,2019-02,150
2,2019-03,198
3,2019-04,202
4,2019-05,214
5,2019-06,176
6,2019-07,210
7,2019-08,201
8,2019-09,197
9,2019-10,186


In [8]:
per_case = []
for i in range(len(month_df)):
    per_case.append((month_df['occur_cnt'][i] / month_damage_df['property_damage_sum_k'][i]) * 100 )
    
per_case

[0.0777082056876429,
 0.0232904788522452,
 0.05257679398394018,
 0.01919416801358415,
 0.04312594589910544,
 0.11678909614529626,
 0.01743255055647192,
 0.07150480256136606,
 0.08269805554622696,
 0.09283753431494884,
 0.03929409317322917,
 0.03248745138628788]

In [9]:
district = [] # 구
district_code = [] # 구 코드
for c in factor['district']:
    if c not in district:
        district.append(c)
for d in factor['district_code']:
    if d not in district_code:
        district_code.append(d)

district_sum = [] # 구 발생건수
for i in range(len(district)):
    d = factor[factor.district == district[i]]   
    ds = d['district'].count()
    district_sum.append(ds)
    
# 구 피해규모
d_damage_sum = []
for i in range(len(district)):
    d = factor[factor.district == district[i]]
    ds = d['property_damage'].sum()
    d_damage_sum.append(ds)
    

print(district, district_code, district_sum, d_damage_sum)

['부산진구', '남구', '사상구', '강서구', '해운대구', '동래구', '동구', '영도구', '기장군', '사하구', '수영구', '서구', '금정구', '중구', '연제구', '북구'] ['D07', 'D04', 'D09', 'D01', 'D16', 'D06', 'D05', 'D14', 'D03', 'D10', 'D12', 'D11', 'D02', 'D15', 'D13', 'D08'] [285, 162, 190, 243, 215, 150, 89, 94, 163, 226, 103, 76, 151, 70, 102, 121] [310881, 237793, 522456, 619501, 589675, 162916, 58212, 179840, 1293514, 630273, 107320, 60253, 700612, 46218, 222869, 500581]


In [32]:
month_district_sum = []
re_month = []
re_district = []

for i in range(len(month)):
    m = month[i]
    for j in range(len(district)):
        d = district[j]
        md = factor[factor.fire_date == m][factor.district == d]
        mds = md['district'].count()
        re_month.append(m)
        re_district.append(d)
        month_district_sum.append(mds)
        
mds_df = pd.DataFrame({
    'month':re_month,
    'district':re_district,
    'occur_cnt':month_district_sum
})

mds_df


,month,district,occur_cnt
0,2019-01,부산진구,31
1,2019-01,남구,20
2,2019-01,사상구,23
3,2019-01,강서구,25
4,2019-01,해운대구,19
...,...,...,...
187,2019-12,서구,17
188,2019-12,금정구,12
189,2019-12,중구,3
190,2019-12,연제구,11


In [52]:
month_occur_max = []
month_district_max= []

for i in range(len(month)):
    m = month[i]
    md = mds_df[mds_df.month == m]['occur_cnt'].max()
    d = mds_df[mds_df.month == m][mds_df.occur_cnt == md]['district']
    month_occur_max.append(md)
    month_district_max.append(d)

df = pd.DataFrame({
    'occur_max':month_occur_max,
    'district_max':month_district_max
})

df

,occur_max,district_max
0,34,"9 사하구 Name: district, dtype: object"
1,26,"16 부산진구 Name: district, dtype: object"
2,27,"32 부산진구 35 강서구 Name: district, dtype: o..."
3,22,"48 부산진구 Name: district, dtype: object"
4,25,"64 부산진구 Name: district, dtype: object"
5,25,"80 부산진구 Name: district, dtype: object"
6,28,"100 해운대구 Name: district, dtype: object"
7,25,"115 강서구 Name: district, dtype: object"
8,23,"128 부산진구 Name: district, dtype: object"
9,20,"147 강서구 Name: district, dtype: object"


In [53]:
month_district_damage_sum = []
re_month = []
re_district = []
re_damage_sum = []

for i in range(len(month)):
    m = month[i]
    for j in range(len(district)):
        d = district[j]
        md = factor[factor.fire_date == m][factor.district == d]
        mds = md['property_damage'].sum()
        re_damage_sum.append(d_damage_sum[i])
        re_month.append(m)
        re_district.append(d)
        month_district_damage_sum.append(mds)
        

        
mdds_df = pd.DataFrame({
    'month':re_month,
    'district':re_district,
    'damage_amount':month_district_damage_sum,
    'month_damage_amount':re_damage_sum
})

mdds_df

,month,district,damage_amount,month_damage_amount
0,2019-01,부산진구,21553,310881
1,2019-01,남구,31907,310881
2,2019-01,사상구,21876,310881
3,2019-01,강서구,25480,310881
4,2019-01,해운대구,12789,310881
...,...,...,...,...
187,2019-12,서구,8233,60253
188,2019-12,금정구,14370,60253
189,2019-12,중구,173,60253
190,2019-12,연제구,6754,60253


In [56]:
month_damage_max = []
month_district_damage_max = []

for i in range(len(month)):
    m = month[i]
    md = mdds_df[mdds_df.month == m]['damage_amount'].max()
    d = mdds_df[mdds_df.month == m][mdds_df.damage_amount == md]['district']
    month_damage_max.append(md)
    month_district_damage_max.append(d)

df = pd.DataFrame({
    'damage_max':month_damage_max,
    'district_damage_max':month_district_damage_max
})

df

,damage_max,district_damage_max
0,53443,"8 기장군 Name: district, dtype: object"
1,375026,"25 사하구 Name: district, dtype: object"
2,126566,"35 강서구 Name: district, dtype: object"
3,402892,"63 북구 Name: district, dtype: object"
4,185689,"66 사상구 Name: district, dtype: object"
5,44497,"81 남구 Name: district, dtype: object"
6,786522,"104 기장군 Name: district, dtype: object"
7,46465,"112 부산진구 Name: district, dtype: object"
8,81138,"140 금정구 Name: district, dtype: object"
9,28745,"148 해운대구 Name: district, dtype: object"


In [ ]:
district_df = pd.DataFrame({
    'district': district,
    'district_code':district_code,
    'occur_cnt' : district_sum
})

d_damage_df = pd.DataFrame({
    'district': district,
    'district_code':district_code,
    'district_property_damage_k': d_damage_sum
})

d_damage_df

In [ ]:
d07 = d_damage_df[d_damage_df.district_code == 'D07']
d07

In [ ]:
# 월별 피해규모 및 지역별 피해규모
fig, axes = plt.subplots(1, 2, figsize=(30,14))
axes[0].bar(month_damage_df['fire_date'], month_damage_df['property_damage_sum_k'])
axes[1].bar(d_damage_df['district'], d_damage_df['district_property_damage_k'])
plt.rc('xtick', labelsize=12)
plt.show()

In [ ]:
# 월별 피해규모
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.xticks(rotation = 30)
for i, v in enumerate(month_df['fire_date']):
    plt.text(v, month_damage_df['property_damage_sum_k'][i], str(month_damage_df['property_damage_sum_k'][i]),
            fontsize=13,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.bar(month_damage_df['fire_date'], month_damage_df['property_damage_sum_k'], color='#95C2B7', edgecolor='mintcream', alpha=0.9, linewidth=8)
plt.title('월별 피해규모', fontsize=20)
plt.show()

In [ ]:
# 지역별 피해규모
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.xticks(rotation = 30)
for i, v in enumerate(d_damage_df['district']):
    plt.text(v, d_damage_df['district_property_damage_k'][i], str(d_damage_df['district_property_damage_k'][i]),
            fontsize=13,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.bar(d_damage_df['district'], d_damage_df['district_property_damage_k'], color='#95C2B7', edgecolor='mintcream', alpha=0.9, linewidth=8)
plt.title('지역별 피해규모', fontsize=20)
plt.show()

In [ ]:
dead_sum = factor['dead'].sum()
injury_sum = factor['injury'].sum()

damage_people = ['사망자', '부상자']
damage_sum = [dead_sum, injury_sum]
wedgeprops = {'width':0.7, 'edgecolor':'w', 'linewidth':5}
pie_colors = ['teal', '#95C2B7']

dp = pd.DataFrame({
    'damage_people':damage_people,
    'damage_sum':damage_sum
})

fig, axes = plt.subplots(1, 2, figsize=(28,14))
plt.rc('font', size=30)
plt.rc('xtick', labelsize=20)
axes[0].pie(damage_sum, autopct='%.2f%%', startangle=250, counterclock=False, labels=damage_people, wedgeprops=wedgeprops, colors=pie_colors)
axes[0].set_title('부산광역시 인명피해 규모', fontsize=20)
axes[1].bar(damage_people, damage_sum, color=pie_colors)
plt.rc('xtick', labelsize=20)
axes[1].set_title('부산광역시 인명피해 규모', fontsize=20)
for i, v in enumerate(damage_people):
    plt.text(v, damage_sum[i], str(damage_sum[i]),
            fontsize=20,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.show()

In [ ]:
plt.rc('font', size=30)
plt.rc('xtick', labelsize=20)
plt.pie(damage_sum, autopct='%.2f%%', startangle=250, counterclock=False, labels=damage_people, wedgeprops=wedgeprops, colors=pie_colors)
plt.title('부산광역시 인명피해 규모', fontsize=20)
plt.show()

In [ ]:
colors = sns.color_palette('hls', len(month_df))
plt.rc('xtick', labelsize=13)
plt.bar(month_df['fire_date'], month_df['occur_cnt'], color=colors, edgecolor=colors, alpha=0.9, linewidth=6)
plt.ylabel('발생건수')
plt.show()

In [ ]:
# 월별 화재건수
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.xticks(rotation = 30)
plt.bar(month_df['fire_date'], month_df['occur_cnt'], color='#95C2B7', edgecolor='mintcream', alpha=0.9, linewidth=8)
for i, v in enumerate(month_df['fire_date']):
    plt.text(v, month_df['occur_cnt'][i], str(month_df['occur_cnt'][i]),
            fontsize=13,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.title('월별 화재건수', fontsize=20)
plt.show()

In [ ]:
# 지역별 화재건수
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.xticks(rotation = 30)
plt.bar(district_df['district'], district_df['occur_cnt'], color='#95C2B7', edgecolor='mintcream', alpha=0.9, linewidth=8)
for i, v in enumerate(district_df['district']):
    plt.text(v, district_df['occur_cnt'][i], str(district_df['occur_cnt'][i]),
            fontsize=13,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.title('구별 화재건수', fontsize=20)
plt.show()

In [ ]:
fd = factor[factor['fire_date']=='2019-03'] # '2019-03' 이 부분을 변경값을 주어져야 함.

sns.countplot(x='district', hue='fire_date', data=fd)
plt.rc('xtick', labelsize=12)

In [ ]:
# 화재유형
fire_type = []
fire_type_code = []
for t in factor['fire_type']:
    if t not in fire_type:
        fire_type.append(t)
        
for t in factor['type_code']:
    if t not in fire_type_code:
        fire_type_code.append(t)
        
type_sum = []
for t in range(len(fire_type)):
    ft = factor[factor.fire_type == fire_type[t]]
    fts = ft['fire_type'].count()
    type_sum.append(fts)
    
    
# 화재원인       
fire_source = []
fire_source_code = []
for s in factor['fire_source']:
    if s not in fire_source:
        fire_source.append(s)
        
for s in factor['fire_source_code']:
    if s not in fire_source_code:
        fire_source_code.append(s)
        
source_sum = []
for s in range(len(fire_source)):
    fs = factor[factor.fire_source == fire_source[s]]
    fss = fs['fire_source'].count()
    source_sum.append(fss)
    
    
# 화재요인
fire_factor = []
fire_factor_code = []
for f in factor['fire_factor']:
    if f not in fire_factor:
        fire_factor.append(f)
    
for f in factor['fire_factor_code']:
    if f not in fire_factor_code:
        fire_factor_code.append(f)    

factor_sum = []
for f in range(len(fire_factor)):
    ff = factor[factor.fire_factor == fire_factor[f]]
    ffs = ff['fire_factor'].count()
    factor_sum.append(ffs)
    
    
type_df = pd.DataFrame({
    'fire_type':fire_type,
    'fire_type_code':fire_type_code,
    'type_sum':type_sum
})

source_df = pd.DataFrame({
    'fire_source':fire_source,
    'fire_source_code':fire_source_code,
    'source_sum':source_sum
})

factor_df = pd.DataFrame({
    'fire_factor':fire_factor,
    'fire_factor_code':fire_factor_code,
    'factor_sum':factor_sum
})


factor_df

In [ ]:
# 화재유형
plt.bar(type_df['fire_type'], type_df['type_sum'], color='#95C2B7', edgecolor='mintcream', alpha=0.9, linewidth=8)
for i, v in enumerate(type_df['fire_type']):
    plt.text(v, type_df['type_sum'][i], str(type_df['type_sum'][i]),
            fontsize=13,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.title('화재유형', fontsize=20)
plt.show()

In [ ]:
# 화재원인
plt.bar(source_df['fire_source'], source_df['source_sum'], color='#95C2B7', edgecolor='mintcream', alpha=0.9, linewidth=8)
for i, v in enumerate(source_df['fire_source']):
    plt.text(v, source_df['source_sum'][i], str(source_df['source_sum'][i]),
            fontsize=13,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.title('화재요인', fontsize=20)
plt.show()

In [ ]:
plt.bar(factor_df['fire_factor'], factor_df['factor_sum'], color='#95C2B7', edgecolor='mintcream', alpha=0.9, linewidth=8)
for i, v in enumerate(factor_df['fire_factor']):
    plt.text(v, factor_df['factor_sum'][i], str(factor_df['factor_sum'][i]),
            fontsize=13,
            color='black',
            horizontalalignment='center',
            verticalalignment='bottom')
plt.title('발화요인', fontsize=20)
plt.show()